# 合并ksrpc+frpc两阻塞代码于单Notebook

In [ ]:
def run_ksrpc():
    from ksrpc import config
    config.PORT = 8000

    from ksrpc.app import simple_
    simple_.main(fork=True)


def run_command(name, args):
    import select
    import subprocess

    try:
        # 使用 subprocess 运行外部命令
        process = subprocess.Popen(
            args,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
        )

        # 实时输出日志
        while process.poll() is None:
            # 检查 stdout/stderr 是否有数据
            rlist, _, _ = select.select([process.stdout, process.stderr], [], [], 0.1)
            for stream in rlist:
                line = stream.readline()
                if line:
                    print(name, "|", line.strip())

        # 等待进程结束
        process.wait()
    except KeyboardInterrupt:
        print(args, "已中断")
    except Exception as e:
        print(args, "启动失败: ", e)
    finally:
        print(args, "已终止")


class ProcessManager:
    def __init__(self, *processes):
        self.processes = processes

    def __enter__(self):
        for p in self.processes:
            p.start()
            print(f"Started process: {p.name} (PID: {p.pid})")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Terminating processes...")
        import os
        import signal

        for p in self.processes:
            if p.is_alive():
                print(f"Terminating {p.name} (PID: {p.pid})")
                p.terminate()  # 发送SIGTERM
                p.join(timeout=3)  # 等待5秒
                if p.is_alive():
                    print(f"Force killing {p.name} (PID: {p.pid})")
                    os.kill(p.pid, signal.SIGKILL)  # 强制终止
                    p.join()
        print("All processes terminated")
        return False  # 不抑制异常


if __name__ == '__main__':
    import multiprocessing
    import time
    from datetime import datetime
    from IPython.display import clear_output

    p1 = multiprocessing.Process(target=run_ksrpc, args=(), name="ksrpc")
    # p1 = multiprocessing.Process(target=run_command, args=('ksrpc', ["python", "-u", "-m", "ksrpc.run_simple"]), name="ksrpc")
    p2 = multiprocessing.Process(target=run_command, args=('frpc', ["./frpc", "-c", "./frpc.toml"]), name="frpc")

    start_time = time.perf_counter()
    try:
        ELAPSE = 60 * 10
        print(f"{datetime.now()} | !!!防止长期占用资源，从开启服务时计算，{round(ELAPSE / 60)}分钟后将退出!!!")
        i = 0
        with ProcessManager(p1, p2):
            while True:
                end_time = time.perf_counter()
                if end_time - start_time > ELAPSE:
                    print(f"{datetime.now()} | !!!{round(ELAPSE / 60)}分钟已到，退出!!!")
                    break
                time.sleep(5)
                i += 1
                if i % 30 == 0:  # 清理可以改成0，不清理改成-1
                    clear_output(wait=True)
    except KeyboardInterrupt:
        print("主进程已中断")
    except Exception as e:
        print("主进程错误: ", e)
    finally:
        print("所有服务已关闭")
        from IPython.display import display, Javascript

        display(Javascript('IPython.notebook.kernel.interrupt()'))
        print("中断内核")